# **SET UP**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import os

from scipy import stats
from scipy.stats import t, linregress
from matplotlib.dates import date2num, num2date
import datetime
from datetime import datetime, date, timedelta
import folium
import branca.colormap
from geopy.geocoders import Nominatim

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# **DataAnalyzer class**

In [ ]:
class DataAnalyzer:
    def __init__(self, data_or_url):
        if isinstance(data_or_url, str):  # Assuming file path is a string
            self.data_url = data_or_url
            self.data = self.load_data()
        elif isinstance(data_or_url, pd.DataFrame):  # Assuming DataFrame
            self.data = data_or_url
        else:
            raise ValueError("Input must be a file path or a pandas DataFrame")

    def load_data(self):
        file_extension = self.data_url.split('.')[-1]
        if file_extension == 'csv':
            return pd.read_csv(self.data_url)
        elif file_extension in ['xlsx', 'xlsm', 'xltx', 'xltm']:  # Add more excel file extensions if needed
            return pd.read_excel(self.data_url)
        else:
            raise ValueError("Unsupported file format")

    def print_column_names(self):
        print("Column names in the dataset:", self.data.columns.tolist())

    def preprocess_data(self, well_name_column, analyte_name_column, well_name, analyte_name):
        filtered_data = self.data[(self.data[well_name_column] == well_name) & (self.data[analyte_name_column] == analyte_name)]
        filtered_data = filtered_data.dropna(subset=['COLLECTION_DATE', 'RESULT'])
        filtered_data['COLLECTION_DATE'] = pd.to_datetime(filtered_data['COLLECTION_DATE']).dt.date
        filtered_data['RESULT_LOG'] = np.log10(filtered_data['RESULT'])
        return filtered_data

    def remove_outliers(self, data, result_column='RESULT_LOG', z_threshold=4):
        z = np.abs(stats.zscore(data[result_column]))
        return data[(z < z_threshold)]

    def predict_time_at_MCL(self, slope, intercept, log_MCL, earliest_date):
        if slope >= 0:
            return None
        time_at_MCL = (log_MCL - intercept) / slope
        try:
            time_at_MCL_date = num2date(time_at_MCL).replace(tzinfo=None).date()
            if earliest_date.year < 1 or time_at_MCL_date.year >= 10000 or time_at_MCL_date <= earliest_date:
                return None
            return time_at_MCL_date
        except ValueError:
            return None

    def adjust_date_based_on_confidence(self, base_date, slope, confidence_interval, days_per_unit):
        if base_date is None or slope == 0:
            return None, None, None
        # Ensure base_date is a date object, not a string or other type
        days_adjustment = confidence_interval / abs(slope) * days_per_unit
        av_date = base_date - timedelta(days=days_adjustment) if base_date else None
        upper_date = base_date + timedelta(days=days_adjustment) if base_date else None
        # Calculate lower_date only if both av_date and upper_date are not None
        lower_date = av_date + (upper_date - av_date) / 2 if av_date and upper_date else None
        return av_date, upper_date, lower_date

    def generate_analyte_summary(self, analyte_name_column, analyte_name, well_name_column, MCL):
        unique_wells = self.data[self.data[analyte_name_column] == analyte_name][well_name_column].unique()
        summary = []

        for well in unique_wells:
            query = self.preprocess_data(well_name_column, analyte_name_column, well, analyte_name)
            if query.empty or len(query['COLLECTION_DATE'].unique()) < 2:
                continue

            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']
            slope, intercept, r_value, p_value, std_err = linregress(x, y)

            if slope > 0:
                trend = 'Increase'
            elif slope < 0:
                trend = 'Decrease'
            else:
                trend = 'Neutral'

            if trend == 'Neutral':
                continue

            # Initialize your variables
            time_at_MCL = None
            total_decrease_duration = None  # This will represent the new av_time@MCL
            upper_date = None
            lower_date = None
            min_time_at_MCL = None
            max_time_at_MCL = None
            mean_time_at_MCL = None

            if trend == 'Decrease':
                log_MCL = np.log10(MCL)
                time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE']))
                min_time_at_MCL = time_at_MCL if isinstance(time_at_MCL, date) else None

                earliest_measurement = min(query['COLLECTION_DATE'])
                if min_time_at_MCL:
                    total_decrease_duration = (min_time_at_MCL - earliest_measurement).days  # Duration in days

                max_concentration_index = query['RESULT'].idxmax()
                max_time_at_MCL = query.loc[max_concentration_index, 'COLLECTION_DATE']

                if min_time_at_MCL and max_time_at_MCL:
                    avg_timestamp = (date2num(max_time_at_MCL) + date2num(min_time_at_MCL)) / 2
                    mean_time_at_MCL = num2date(avg_timestamp).date()

            # Calculate the confidence interval adjustments only if you have time_at_MCL and it's a date
            df = len(x) - 2
            ci_slope = std_err * t.ppf(1 - 0.025, df) if df > 0 else 0
            if time_at_MCL and isinstance(time_at_MCL, date):
                _, upper_date, lower_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ci_slope, 365.25)

            summary.append({
                'well_name': well,
                'slope': slope,
                'trend': trend,
                'time@MCL': time_at_MCL,
                'total_decrease_duration': total_decrease_duration,  # Renamed from av_time@MCL
                'upper_time@MCL': upper_date,
                'lower_time@MCL': lower_date,
                'min_time@MCL': min_time_at_MCL,
                'max_time@MCL': max_time_at_MCL,
                'mean_time_at_MCL': mean_time_at_MCL,
                'slope_confidence_interval': ci_slope,
                'intercept_confidence_interval': intercept if slope else None,
            })

        summary_df = pd.DataFrame(summary)
        return summary_df

    def compile_results(self, well_analyte_pairs, well_name_column, analyte_name_column, MCL):
        results = []
        for well_name, analyte_name in well_analyte_pairs:
            time_at_MCL = None
            query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
            if query.empty:
                continue

            query = self.remove_outliers(query)
            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']

            if len(x) < 2:
                continue

            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            df = len(x) - 2
            tinv = lambda p, df: abs(t.ppf(p/2, df))
            ts = tinv(0.05, df)

            trend = "Increase" if slope > 0 else "Decrease" if slope < 0 else "Neutral"
            log_MCL = np.log10(MCL) if MCL else None
            earliest_date = min(query['COLLECTION_DATE'])
            time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, earliest_date) if MCL else None

            results.append({
                'well_name': well_name,
                'slope': slope,
                'trend': trend,
                'time@MCL': time_at_MCL,
                'slope_confidence_interval': ts * std_err,
                'intercept_confidence_interval': ts * intercept
            })

        results_df = pd.DataFrame(results)
        results_df.to_csv('time2MCL_results.csv', index=False)
        return results_df

    def plot_MCL(self, well_name_column, analyte_name_column, well_name, analyte_name, year_interval=5, save_dir='plot_MCL', MCL=None, extrapolate_until_year=2025):
        query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
        if query.empty:
            print(f'No results found for {well_name} and {analyte_name}')
            return

        query = self.remove_outliers(query, result_column='RESULT_LOG')
        x = date2num(query['COLLECTION_DATE'])
        y = query['RESULT_LOG']

        if len(x) < 2:  # Not enough data points to perform regression
            print("Not enough data for regression.")
            return

        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        df = len(x) - 2  # degrees of freedom for t-distribution
        tinv = lambda p, df: abs(stats.t.ppf(p/2, df))  # two-sided t-test
        ts = tinv(0.05, df)  # 95% confidence interval multiplier
        slope_confidence_interval = ts * std_err

        log_MCL = np.log10(MCL) if MCL else None
        time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE'])) if MCL and slope < 0 else 'Not available'

        # Prepare for extrapolation
        extrapolate_date = np.datetime64(f'{extrapolate_until_year}-12-31')
        x_extrapolate = np.linspace(x.min(), date2num(extrapolate_date), 100)
        y_extrapolate = slope * x_extrapolate + intercept

        # Categorize and label the trend based on the slope
        trend_label = 'Increase (+)' if slope > 0 else 'Decrease (-)' if slope < 0 else 'Neutral (/)'

        plt.figure(figsize=(10, 6))
        plt.scatter(query['COLLECTION_DATE'], y, label='Data Points')
        for i, point in query.iterrows():
            plt.text(point['COLLECTION_DATE'], point['RESULT_LOG'], str(i), fontsize=6, ha='center', va='center')
        plt.plot(num2date(x_extrapolate), y_extrapolate, 'r--', label=f'Extrapolated Regression ({trend_label})')

        if log_MCL is not None:
            plt.axhline(y=log_MCL, color='green', linestyle='--', label=f'MCL (log10): {log_MCL:.2f}')

        plt.figtext(0.15, 0.05, f"Slope: {slope:.2e} +/- {slope_confidence_interval:.2e}\nTime@MCL: {time_at_MCL}", fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

        plt.xlabel('Collection Date')
        plt.ylabel('Log-Concentration (pCi/mL)')
        plt.title(f'{well_name} - {analyte_name}')
        plt.legend()

        plt.gca().xaxis.set_major_locator(mdates.YearLocator(year_interval))
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gcf().autofmt_xdate()

        plt.xlim([num2date(x.min()), num2date(x_extrapolate.max())])

        figure_path = f'{save_dir}/{well_name}-{analyte_name}.png'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(figure_path)
        #plt.close()  # Close the plot after saving to avoid displaying it
        return figure_path

    def generate_analysis_dataframe(self, analyte_name_column, well_name_column, analyte_name, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        analysis_data = []

        for index, row in analyte_summary_df.iterrows():
            if row['trend'] == 'Neutral':
                continue

            plot_path = self.plot_MCL(well_name_column, analyte_name_column, row['well_name'], analyte_name, year_interval=5, save_dir='plot_MCL', MCL=MCL)

            analysis_data.append({
                'analyte_name': analyte_name,
                'well_name': row['well_name'],
                'slope': row['slope'],
                'trend': row['trend'],
                'total_decrease_duration': row['total_decrease_duration'],
                'upper_time@MCL': row['upper_time@MCL'],
                'lower_time@MCL': row['lower_time@MCL'],
                'path_to_figure': plot_path
            })

        return pd.DataFrame(analysis_data)

    def generate_decreasing_well_list(self, analyte_name_column, analyte_name, well_name_column, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        decreasing_well_list = analyte_summary_df[analyte_summary_df['trend'] == 'Decrease']['well_name'].tolist()
        return decreasing_well_list

    def extract_location_data(self, well_name_column, decreasing_well_list):
        location_data = self.data[self.data[well_name_column].isin(decreasing_well_list)]
        return location_data

    def forecast_average_concentration(self, analyte_name_column, well_name_column, year, analyte_name, MCL):
        decreasing_wells = self.generate_decreasing_well_list(analyte_name_column, analyte_name, well_name_column, MCL)

        forecasted_concentration = []

        unique_wells = self.data[self.data[well_name_column].isin(decreasing_wells)][well_name_column].unique()

        for well in unique_wells:
            query = self.preprocess_data(well_name_column, analyte_name_column, well, analyte_name)
            if query.empty or len(query['COLLECTION_DATE'].unique()) < 2:
                continue

            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']

            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            forecasted_log_concentration = slope * date2num(pd.to_datetime(f'{year}-01-01')) + intercept

            # Create the custom key for the predicted concentration
            concentration_key = f"concentration_of_{analyte_name}_in_the_year_{year}"

            total_decrease_duration = None
            if slope < 0:
                log_MCL = np.log10(MCL)
                time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE']))
                if time_at_MCL:
                    total_decrease_duration = (time_at_MCL - min(query['COLLECTION_DATE'])).days

            forecasted_concentration.append({
                'STATION_ID': well,
                concentration_key: 10**forecasted_log_concentration,  # Convert log concentration back to original scale
                'total_decrease_duration': total_decrease_duration
            })

        forecasted_df = pd.DataFrame(forecasted_concentration)
        return forecasted_df

    def merge_with_location_data(self, location_data, average_concentration):
        # Ensure both keys are strings
        location_data['station_id'] = location_data['station_id'].astype(str).str.strip().str.upper()
        average_concentration['STATION_ID'] = average_concentration['STATION_ID'].astype(str).str.strip().str.upper()

        # Identify common wells
        common_wells = set(location_data['station_id']).intersection(set(average_concentration['STATION_ID']))

        # Filter location_data to retain only common wells
        filtered_location_data = location_data[location_data['station_id'].isin(common_wells)]

        # Merge the data
        merged_data = pd.merge(filtered_location_data, average_concentration, left_on='station_id', right_on='STATION_ID', how='left')

        # Rename the 'total_decrease_duration' column to 'time@MCL'
        if 'total_decrease_duration' in merged_data.columns:
            merged_data.rename(columns={'total_decrease_duration': 'time@MCL'}, inplace=True)

        return merged_data

In [ ]:
data_analyzer = DataAnalyzer('https://raw.githubusercontent.com/ALTEMIS-DOE/pylenm/master/notebooks/data/FASB_Data_thru_3Q2015_Reduced_Demo.csv')
location_analyzer = DataAnalyzer('/content/gdrive/MyDrive/MIT Project - VAL @HW_CEE/FASB Well Construction Info.xlsx')

# **Gather variables for calculation**

In [ ]:
average_concentration_2024_sr_90 = data_analyzer.forecast_average_concentration(analyte_name_column='ANALYTE_NAME', well_name_column='STATION_ID', year=2024, analyte_name='STRONTIUM-90', MCL=10)

In [ ]:
print(average_concentration_2024_sr_90)

In [ ]:
average_concentration_2024_i_129 = data_analyzer.forecast_average_concentration(analyte_name_column='ANALYTE_NAME', well_name_column='STATION_ID', year=2024, analyte_name='IODINE-129', MCL=10)

In [ ]:
print(average_concentration_2024_i_129)

In [ ]:
location_data = location_analyzer.data
location_data['SZ_AV'] = (location_data['SZ_TOP(ft MSL)'] + location_data['SZ_BOT(ft MSL)']) / 2
location_data['SZ_window'] = location_data['SZ_TOP(ft MSL)'] - location_data['SZ_BOT(ft MSL)']

In [ ]:
print(location_data[['SZ_TOP(ft MSL)', 'SZ_BOT(ft MSL)', 'SZ_AV', 'SZ_window']].head())

In [ ]:
merged_data_sr_90 = data_analyzer.merge_with_location_data(location_data, average_concentration_2024_sr_90)
merged_data_i_129 = data_analyzer.merge_with_location_data(location_data, average_concentration_2024_i_129)

In [ ]:
print(merged_data_sr_90)

In [ ]:
print(merged_data_sr_90.columns)

In [ ]:
merged_data_sr_90 = merged_data_sr_90.rename(columns={
    'concentration_of_STRONTIUM-90_in_the_year_2024': 'Sr_90_initial_concentration',
    'SZ_AV': 'screen_zone_average',
    'SZ_window': 'screen_zone_window',
})

In [ ]:
merged_data_sr_90['aquifer'] = merged_data_sr_90['aquifer'].replace({
    'UAZ_UTRAU': 'UUTRA',
    'LAZ_UTRAU': 'LUTRA'
})

In [ ]:
print(merged_data_i_129)

In [ ]:
print(merged_data_i_129.columns)

In [ ]:
merged_data_i_129 = merged_data_i_129.rename(columns={
    'concentration_of_IODINE-129_in_the_year_2024': 'I_129_initial_concentration',
    'SZ_AV': 'screen_zone_average',
    'SZ_window': 'screen_zone_window',
})

In [ ]:
merged_data_i_129['aquifer'] = merged_data_i_129['aquifer'].replace({
    'UAZ_UTRAU': 'UUTRA',
    'LAZ_UTRAU': 'LUTRA'
})

# **Random forest regression**

In [ ]:
!pip install scienceplots

In [ ]:
import scienceplots

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def random_forest_regression(merged_data, target_column, feature_columns, categorical_column=None):
    # Data preparation (same as before)
    for col in feature_columns:
        if categorical_column is None or col != categorical_column:
            merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')

    merged_data[target_column] = pd.to_numeric(merged_data[target_column], errors='coerce')
    merged_data = merged_data.dropna(subset=[target_column])

    if categorical_column is not None:
        merged_data[categorical_column] = merged_data[categorical_column].astype('category')
        categorical_data = pd.get_dummies(merged_data[categorical_column], drop_first=True)
        merged_data = pd.concat([merged_data[feature_columns].drop(columns=[categorical_column]), categorical_data, merged_data[target_column]], axis=1)
    else:
        merged_data = merged_data[feature_columns + [target_column]]

    # Drop rows with missing values
    merged_data = merged_data.dropna()

    X = merged_data.drop(columns=[target_column])
    y = merged_data[target_column]

    # Ensure enough data
    if X.shape[0] < 2:
        print("Not enough data for training. Ensure there are more samples.")
        return

    # Use a pipeline to standardize the data and apply RandomForestRegressor
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
    ])

    # Cross-validation
    cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')
    print(f'Cross-validated R^2 scores: {cv_scores}')
    print(f'Mean R^2 score: {cv_scores.mean()}')

    # Train the model
    pipeline.fit(X, y)
    y_pred = pipeline.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    print(f'Mean Squared Error: {mse}')
    print(f'R^2 Score: {r2}')

    # Feature importance
    rf = pipeline.named_steps['rf']
    feature_importances = pd.Series(rf.feature_importances_, index=X.columns)
    feature_importances = feature_importances.sort_values(ascending=False)

    print("Feature Importances:")
    print(feature_importances)

    return pipeline, feature_importances, r2, mse

In [ ]:
def plot_feature_importances(feature_importances, analyte, concentration_status):
    plt.style.use(['science', 'bright'])
    plt.rcParams.update({
        "text.usetex": False,
        "font.size": 16
    })
    fig, ax = plt.subplots(figsize=(12, 8))

    colors = plt.colormaps.get_cmap('tab10')(np.linspace(0, 1, len(feature_importances)))
    explode = [0.05 if importance < 0.05 else 0 for importance in feature_importances]
    wedges, texts = ax.pie(
        feature_importances, labels=None, colors=colors, startangle=90, explode=explode
    )
    distance_levels = np.linspace(1.00, 1.2, len(feature_importances))  # Bring labels closer to the chart
    label_shift_factor = 0.035  # Minimize the separation between labels

    for i, wedge in enumerate(wedges):
        angle = (wedge.theta2 + wedge.theta1) / 2  # Middle angle of the wedge
        x = np.cos(np.deg2rad(angle)) * (distance_levels[i] + i * label_shift_factor)
        y = np.sin(np.deg2rad(angle)) * (distance_levels[i] + i * label_shift_factor)

        ax.text(x, y, f'{feature_importances[i]*100:.1f}%', ha='center', fontsize=14, color='black')

    feature_labels = feature_importances.index
    handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in colors]
    legend = ax.legend(handles, feature_labels, title="Features", loc="center left", bbox_to_anchor=(1, 0.5))

    fig.subplots_adjust(top=0.85, right=0.75)

    # Modify the title to include 'with' or 'without' based on the argument
    fig.suptitle(f'Feature Importances for {analyte}\n{concentration_status} the analyte concentration', fontsize=18, x=0.9, y=0.92)

    ax.axis('equal')

    plt.show()

In [ ]:
categorical_column_1 = 'aquifer'
target_column = 'time@MCL'

In [ ]:
# Sr-90 with analyte concentration
feature_columns_1 = [
    'Sr_90_initial_concentration',
    'screen_zone_average', 'screen_zone_window', 'latitude', 'longitude',
    'easting', 'northing', 'ground_elevation',
]

pipeline_sr_90_1, feature_importances_sr_90_1, r2_sr90_with, mse_sr90_with = random_forest_regression(
    merged_data_sr_90, target_column, feature_columns_1
)

if feature_importances_sr_90_1 is not None:
    plot_feature_importances(feature_importances_sr_90_1, 'Sr-90', 'with')

In [ ]:
# Sr-90 without analyte concentration
feature_columns_3 = [
    'screen_zone_average', 'screen_zone_window', 'latitude', 'longitude',
    'easting', 'northing', 'ground_elevation',
]

pipeline_sr_90_2, feature_importances_sr_90_2, r2_sr90_without, mse_sr90_without = random_forest_regression(
    merged_data_sr_90, target_column, feature_columns_3
)

if feature_importances_sr_90_2 is not None:
    plot_feature_importances(feature_importances_sr_90_2, 'Sr-90', 'without')

In [ ]:
# Prepare variables for I-129 with analyte concentration
feature_columns_2 = [
    'I_129_initial_concentration',
    'screen_zone_average', 'screen_zone_window', 'latitude', 'longitude',
    'easting', 'northing', 'ground_elevation',
]

pipeline_i_129_1, feature_importances_i_129_1, r2_i129_with, mse_i129_with = random_forest_regression(
    merged_data_i_129, target_column, feature_columns_2
)

if feature_importances_i_129_1 is not None:
    plot_feature_importances(feature_importances_i_129_1, 'I-129', 'with')

In [ ]:
# I-129 without analyte concentration
feature_columns_4 = [
    'screen_zone_average', 'screen_zone_window', 'latitude', 'longitude',
    'easting', 'northing', 'ground_elevation',
]

pipeline_i_129_2, feature_importances_i_129_2, r2_i129_without, mse_i129_without = random_forest_regression(
    merged_data_i_129, target_column, feature_columns_4
)

if feature_importances_i_129_2 is not None:
    plot_feature_importances(feature_importances_i_129_2, 'I-129', 'without')

In [ ]:
from matplotlib import cm
import matplotlib.pyplot as plt

In [ ]:
# Enable science style from SciencePlots package
plt.style.use(['science', 'bright'])

# Disable LaTeX rendering for text elements
plt.rcParams.update({
    "text.usetex": False,      # Disable LaTeX
    "font.size": 16            # Increase font size
})

# Prepare data
def get_performance_metrics(pipeline, X, y):
    y_pred = pipeline.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    return r2, mse

# Data for plotting
metrics = ['R² Score', 'Mean Squared Error']
sr90_with = [r2_sr90_with, mse_sr90_with]
sr90_without = [r2_sr90_without, mse_sr90_without]
i129_with = [r2_i129_with, mse_i129_with]
i129_without = [r2_i129_without, mse_i129_without]

# Set-up
labels = ['R² Score', 'Mean Squared Error']
groups = ['Sr-90 with Analyte', 'Sr-90 without Analyte', 'I-129 with Analyte', 'I-129 without Analyte']
colors = cm.jet(np.linspace(0, 1, len(groups)))

# Create a bar chart for R² Score
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(groups, [r2_sr90_with, r2_sr90_without, r2_i129_with, r2_i129_without], color=colors)
ax.set_ylabel('R² Score')
ax.set_title('R² Score by Model and Analyte')
plt.xticks(rotation=45, ha='right')

# Improve layout
fig.tight_layout()

# Display the plot
plt.show()

# Create a bar chart for Mean Squared Error
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(groups, [mse_sr90_with, mse_sr90_without, mse_i129_with, mse_i129_without], color=colors)
ax.set_ylabel('Mean Squared Error')
ax.set_title('Mean Squared Error by Model and Analyte')
plt.xticks(rotation=45, ha='right')

# Improve layout
fig.tight_layout()

# Display the plot
plt.show()

# **CF using analytical methods**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import t
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

In [ ]:
def analytical_prediction_intervals(pipeline, X, y, alpha=0.05):
    # Fit the model
    pipeline.fit(X, y)

    # Predictions
    y_pred = pipeline.predict(X)

    # Residuals
    residuals = y - y_pred

    # Degrees of freedom
    n = len(y)
    p = X.shape[1]
    dof = n - p - 1

    # Mean Squared Error
    mse = mean_squared_error(y, y_pred)

    # Standard Error
    try:
        X = X.astype(np.float64)  # Ensure all values in X are float64
        XTX_inv = np.linalg.inv(np.dot(X.T, X))
        se = np.sqrt(mse * (1 + np.diagonal(np.dot(X, np.dot(XTX_inv, X.T)))))
    except np.linalg.LinAlgError:
        se = np.full_like(y_pred, np.nan)  # Set SE to NaN if the matrix is singular
    except ValueError:
        se = np.full_like(y_pred, np.nan)  # Set SE to NaN if conversion fails

    # t-value for the confidence interval
    t_value = t.ppf(1 - alpha / 2, dof)

    # Prediction Interval
    lower_bound = y_pred - t_value * se
    upper_bound = y_pred + t_value * se

    return lower_bound, upper_bound, y_pred

ci_results = {}

datasets = [
    ('Sr-90 with analyte concentration', RandomForestRegressor(), merged_data_sr_90, feature_columns_1),
    ('Sr-90 without analyte concentration', RandomForestRegressor(), merged_data_sr_90, feature_columns_3),
    ('I-129 with analyte concentration', RandomForestRegressor(), merged_data_i_129, feature_columns_2),
    ('I-129 without analyte concentration', RandomForestRegressor(), merged_data_i_129, feature_columns_4)
]

for name, model, data, feature_columns in datasets:
    X = data[feature_columns]
    y = data[target_column]

    # Ensure categorical encoding
    categorical_column = 'aquifer'  # Assuming 'aquifer' is the categorical column
    if categorical_column in X.columns:
        X = pd.get_dummies(X, columns=[categorical_column], drop_first=True)

    # Ensure all data is numerical and handle NaNs
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # Drop rows with NaN values in X or y
    valid_mask = ~X.isna().any(axis=1) & ~y.isna()
    X = X[valid_mask]
    y = y[valid_mask]

    # Use StandardScaler and LinearRegression for the analytical approach
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LinearRegression())
    ])

    lower_bound, upper_bound, predictions = analytical_prediction_intervals(pipeline, X, y)
    ci_results[name] = {'lower_bound': lower_bound, 'upper_bound': upper_bound, 'predictions': predictions}

# Generating the tables for each trial with well names included, converting day units to year units, and adding confidence interval as percentage
for name, ci_data in ci_results.items():
    data_for_current_trial = datasets[[ds[0] for ds in datasets].index(name)][2]  # Get the dataset corresponding to the current trial
    well_names = data_for_current_trial['STATION_ID']

    X = data_for_current_trial[feature_columns]
    y = data_for_current_trial[target_column]

    # Ensure categorical encoding
    categorical_column = 'aquifer'  # Assuming 'aquifer' is the categorical column
    if categorical_column in X.columns:
        X = pd.get_dummies(X, columns=[categorical_column], drop_first=True)

    # Ensure all data is numerical and handle NaNs
    X = X.apply(pd.to_numeric, errors='coerce')
    y = pd.to_numeric(y, errors='coerce')

    # Drop rows with NaN values in X or y
    valid_mask = ~X.isna().any(axis=1) & ~y.isna()
    well_names = well_names[valid_mask]  # Apply valid_mask to well_names
    X = X[valid_mask]
    y = y[valid_mask]

    # Convert results from days to years
    predictions_in_years = ci_data['predictions'] / 365.25
    lower_bound_in_years = ci_data['lower_bound'] / 365.25
    upper_bound_in_years = ci_data['upper_bound'] / 365.25
    confidence_interval_in_years = (ci_data['upper_bound'] - ci_data['lower_bound']) / 365.25
    confidence_interval_percentage = (confidence_interval_in_years / predictions_in_years) * 100

    # Replace negative values with '[0]'
    predictions_in_years = np.where(predictions_in_years < 0, '[0]', predictions_in_years)
    lower_bound_in_years = np.where(lower_bound_in_years < 0, '[0]', lower_bound_in_years)
    upper_bound_in_years = np.where(upper_bound_in_years < 0, '[0]', upper_bound_in_years)
    confidence_interval_in_years = np.where(confidence_interval_in_years < 0, '[0]', confidence_interval_in_years)
    confidence_interval_percentage = np.where(confidence_interval_percentage < 0, '[0]', confidence_interval_percentage)

    table = pd.DataFrame({
        'Station ID': well_names,
        'Prediction (in years)': predictions_in_years,
        'Lower Bound (in years)': lower_bound_in_years,
        'Upper Bound (in years)': upper_bound_in_years,
        'Confidence Interval (in years)': confidence_interval_in_years,
        'Confidence Interval (%)': confidence_interval_percentage
    })
    print(f'Table for {name}:')
    print(table)
    table.to_csv(f'{name}_confidence_intervals_in_years.csv', index=False)